<a href="https://colab.research.google.com/github/jamestheengineer/data-science-from-scratch-Python/blob/master/Chapter_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Getting data

# You can pipe data using stdin and stdout

# egrep.py
import sys, re

# sys.argv is the list of command-line arguments
# sys.argv[0] is the name of the program itself
# sys.argv[1] will be the regex specified at the command line
regex = sys.argv[1]

# for every line passed into the script
for line in sys.stdin:
  # if it matches the regex, write it to stdout
  if re.search(regex, line):
    sys.stdout.write(line)


In [0]:
# line_count.py
count = 0
for line in sys.stdin:
  count += 1

# print goes to sys.stdout
print(count)

In [0]:
# If I were to break these files out, you could then pipe like:
# type SomeFile.txt | python egrep.py "[0-9]" | python line_count.py

In [0]:
# another potential script
# most_common_words.py
import sys
from collections import Counter

# pass in number of words as first argument
try:
  num_words : int(sys.argv[1])
except:
  print("usage: most_common_words.py num_words")
  sys.exit(1) # nonzero exit code indicates error

counter = Counter(word.lower()                      # lowercase words
                  for line in sys.stdin
                  for word in line.strip().split()  # split on spaces
                  if word)                          # skip empty 'words'

for word, count in counter.most_common(num_words):
  sys.stdout.write(str(count))
  sys.stdout.write("\t")
  sys.stdout.write(word)
  sys.stdout.write("\n")
  

In [0]:
# Then you could do
# cat the_bible.txt | python most_common_words.py 10

In [0]:
# Reading Files
